In [1]:
import pandas as pd
import pdfplumber
import os
import numpy as np

pdf_input_path = r'C:\\Users\\azona\\OneDrive - Cerity Partners\\Portfolio Management Team\\2 - Monthly AUM and Performance\\DEV\\2.2 - Scripts\\2 - SCRIPT - Orion Holdings and Performance information\\2._.1 - Fixed Income Composite Standard Rerport PDF'
pdf_file_name = 'CompositeStandardReport_CP-2.pdf'

excel_input_path = r'C:\\Users\\azona\\OneDrive - Cerity Partners\\Portfolio Management Team\\2 - Monthly AUM and Performance\DEV\\2.2 - Scripts\\2 - SCRIPT - Orion Holdings and Performance information\\2._.2 - Orion Group Totals Excel'
excel_file_name = 'Orion Group Nov YTD Totals(Sheet1).csv'

output_path = r'C:\\Users\\azona\\OneDrive - Cerity Partners\\Portfolio Management Team\\2 - Monthly AUM and Performance\\DEV\\2.2 - Scripts\\2 - SCRIPT - Orion Holdings and Performance information\\2 - Output'

os.chdir(pdf_input_path)

In [2]:
with pdfplumber.open(pdf_file_name) as pdf:
    page = pdf.pages[0]
    text = page.extract_text()
    lines = text.split('\n')
    data_rows = []
    capture = False
    
    for line in lines:
        if 'Management Style' in line and 'Period' in line and 'QTD' in line:
            capture = True
            continue
        
        if 'Disclaimer' in line:
            break
        
        if capture and line.strip() and '%' in line:
            parts = line.split()
            first_percent_idx = None
            
            for i, part in enumerate(parts):
                if '%' in part:
                    first_percent_idx = i
                    break
            
            if first_percent_idx:
                name = ' '.join(parts[:first_percent_idx-1])
                data_parts = parts[first_percent_idx-1:]
                values = []
                i = 0
                
                while i < len(data_parts):
                    if i < len(data_parts) - 1 and data_parts[i+1] == '%':
                        values.append(data_parts[i])
                        i += 2
                    elif data_parts[i] == '%':
                        i += 1
                    else:
                        values.append(data_parts[i])
                        i += 1
                
                if len(values) >= 9:
                    row = [name] + values[:9]
                    data_rows.append(row)
    
    columns = ['Management Style', 'Period', 'QTD', 'YTD', 'Rolling 1-Year', 
               'Rolling 3-Year', 'Rolling 5-Year', 'Rolling 10-Year', 
               'Inception', 'Inception Date']
    
    pdf_data = pd.DataFrame(data_rows, columns=columns)
    pdf_data = pdf_data.sort_values('Management Style')
    #data.to_excel('management_styles_report.xlsx', index=False)
    
    print("PDF to Dataframe complete")

# list of columns to drop
pdf_columns_2_drop = ['QTD', 'Rolling 5-Year', 'Rolling 10-Year', 'Inception', 'Inception Date']

# dropping the unused columns
pdf_data = pdf_data.drop(pdf_columns_2_drop, axis=1)

PDF to Dataframe complete


In [3]:
pdf_data.columns

Index(['Management Style', 'Period', 'YTD', 'Rolling 1-Year',
       'Rolling 3-Year'],
      dtype='object')

In [4]:
# change directory to import the excel file
os.chdir(excel_input_path)

# setting variable for excel file
excel_data = pd.read_csv(excel_file_name)

In [5]:
# drop first column as it does not have anything in it that is important
excel_data = excel_data.iloc[1:]

# dropping column 5
# might want to drop column 6 at some point too
excel_data = excel_data.drop( ['Unnamed: 5' , 'Unnamed: 6'], axis=1)

# renaming column 1
excel_data = excel_data.rename(columns={excel_data.columns[0]: 'Team Name'})


In [6]:
print(len(pdf_data))
print(len(excel_data))

35
34


In [7]:
# joining the data 
data_merged = pd.merge(excel_data, pdf_data, left_on='Team Name', right_on='Management Style', how='outer')

In [8]:
data_merged['Organization'] = np.where(
    data_merged['Team Name'] == data_merged['Management Style'], 
    data_merged['Team Name'],
    np.where(pd.isna(data_merged['Team Name']), data_merged['Management Style'], data_merged['Team Name'])
)

In [9]:
columns_sequence = ['Organization', 'Team Name', 'Management Style', 'Starting Value (12/31/2024)', 'Net Cash Flow', 'Total Return', 
                    'Ending Value (11/30/2025)', 'Period', 'YTD', 'Rolling 1-Year', 'Rolling 3-Year']
data_merged = data_merged[columns_sequence]

In [13]:
# Establishing output path locations
output_file = os.path.join(output_path, "Orion Holdings and Performance.xlsx")

# Publish extract
data_merged.to_excel(output_file, sheet_name='sheet1', index=False)